In [ ]:
from ccmm.utils.utils import average_models


for ref_symbol in symbols:
    ref_model = copy.deepcopy(models[ref_symbol])

    all_models_permuted_to_ref = {symb: models_permuted_pairwise[ref_symbol][symb] for symb in symbols}

    model_params = {symbol: model.state_dict() for symbol, model in all_models_permuted_to_ref.items()}

    mean_params = average_models(model_params)

    ref_model.load_state_dict(mean_params)

    results = trainer.test(ref_model, test_loader, verbose=True)